### ETL

load a cvs file

transform - get one column, and group by to get MAX and AVG

load to parquet file



The csv file has the next columns

singer_id|singer_name| concert_name|  duration


Column duration is the time in hours the concert.



Write pyscpark code to do and ETL.

### Tips:
In Transform step,

per Singer, get the maximum duration time of her concerts, and the average time of her concerts. 


columns max_duration and avg_duration must be roud to 2 decimals

output example:

df.show()


| singer_id  | max_duration | avg_duration | 
|---|---|---|
|        1|        3.74|        2.33|
|        3|        3.96|        2.99|
|        2|        3.85|        2.42|



In [7]:
from IPython.display        import display, HTML

import findspark
findspark.init()
import pyspark

#from pyspark.context import SparkContext
from pyspark.sql            import SparkSession
from pyspark.sql.functions  import col, explode, udf
from pyspark.sql.types      import Row, StringType, LongType, ArrayType



In [8]:
spark = SparkSession.builder \
        .appName( 'singers' )  \
        .getOrCreate()

### extract

In [9]:
input_file = '/home/art/data/tiny/singers.csv'

df = spark.read \
    .options( header = True, inferSchema = True, delimiter = ','  ) \
    .csv( input_file )

In [10]:
df.show()

+---------+-----------+-------------+----------+
|singer_id|singer_name| concert_name|  duration|
+---------+-----------+-------------+----------+
|        1| Katy Parry|   World Tour|1.23456789|
|        2|   Doja Cat|  Hip Hop You| 1.3654987|
|        1| Katy Parry|  Animal life|2.02789798|
|        2|   Doja Cat|      Friends|2.03987987|
|        3|    Cardi B|   Sweet lips|2.00987987|
|        1| Katy Parry|Beach and sun|  3.741741|
|        2|   Doja Cat|   Night life|  3.852852|
|        3|    Cardi B| In the party|  3.963963|
+---------+-----------+-------------+----------+



In [11]:
df.printSchema()

root
 |-- singer_id: integer (nullable = true)
 |-- singer_name: string (nullable = true)
 |-- concert_name: string (nullable = true)
 |-- duration: double (nullable = true)



### transform

In [12]:
df.createOrReplaceTempView( 'df' )


In [13]:
query = '''
select singer_id, 
       round( max( duration ), 2 ) as max_duration,
       round( avg( duration ), 2 ) as avg_duration
       
from df
group by 1
'''

df2 = spark.sql( query )

df2.show()


+---------+------------+------------+
|singer_id|max_duration|avg_duration|
+---------+------------+------------+
|        1|        3.74|        2.33|
|        3|        3.96|        2.99|
|        2|        3.85|        2.42|
+---------+------------+------------+



### load

 save to parquet

In [14]:
output_file = '/home/art/data/tiny/singers.parquet'

df2.write.parquet( output_file )